In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

# The target URL
url = 'https://qmrawiki.org/node/402'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find elements by HTML tags, attributes, or CSS class
    titles = soup.find_all('h1')
    for title in titles:
        print(title.get_text())
    tables_data = soup.find_all('table')
    
    for table_data in tables_data[3]:
        table_row_data = [row for row in table_data.get_text().split('\n') if row != '']
        if len(table_row_data):
            columns = table_row_data[1:5]
            experiment_data = []
            print(columns)
            i = 5
            while i <= len(table_row_data):
                experiment_data
                row_data = table_row_data[i:i+4]
                i+=4 
                if(len(row_data) > 0):
                    experiment_data.append(row_data)
    df = pd.DataFrame(columns=columns, data=experiment_data)
    print(df)         
else:
    print('Failed to retrieve the webpage')



Optimization Output for Exp. 183
['Dose', 'Infected', 'Non-infected', 'Total']
   Dose Infected Non-infected Total
0   100        2            2     4
1   300        2            3     5
2  1000        1            2     3
3  3000        3            1     4


In [12]:
url_new = 'https://qmrawiki.org/experiments/cryptosporidium-parvum'

response = requests.get(url_new)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    experiments_container = soup.find(class_='view-pathogen-dose-response-experiments')
    
    #experiments = experiments_container.find_all(class_='response-table-inner')

    #for experiment in experiments:
        #print(experiment.get_text().split(' '))
        
    

['\n\n108', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'oral', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', '8', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Iowa', 'strain', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'oocysts', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'human', '', '', '', '', '', '', '', '', '', '', '', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'exponential', '', '', '', '', '', '', '', '', '', '', '', '\n\n\nk', '=', '4.19E-03\nLD50/ID50', '=', '1.65E+02', '\n', '\n\n', '', '', '', '', '', '', '', '', '', '', '', '', '', 'infection', '', '', '', '', '', '', '', '', '', '', '', '\n\nDuPont,', 'H', 'L.,', 'et', 'al.', '"The', 'infectivity', 'of', 'Cryptosporidium', 'parvum', 'in', 'healthy', 'vol

# Step 1, iterating over all the dose response pages and going to next page

In [2]:
dose_responses_url = 'https://qmrawiki.org/framework/dose-response/experiments'
response = requests.get(dose_responses_url)

In [3]:
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    

In [75]:
individual_dose_responses_list = soup.find(class_='quicktabs-views-group')

dose_response_url = individual_dose_responses_list.find_all('span')
base_url = 'https://qmrawiki.org'
for dose_responses in dose_response_url:
    temps = dose_responses.find_all('a')
    for temp in temps:
        experiment_overview_url = temp.get('href')
        extract_from_dose_response_page('https://qmrawiki.org/experiments/naegleria-fowleri', base_url)
        break

['Dose', 'Dead', 'Survived', 'Total', '2.5E+06', '4', '6', '10', '5E+06', '19', '1', '20', '1E+07', '10', '0', '10']
[['Dose', 'Dead', 'Survived', 'Total'], ['2.5E+06', '4', '6', '10'], ['5E+06', '19', '1', '20'], ['1E+07', '10', '0', '10']]
test
k = 4.21E-07
4.21E-07


AttributeError: 'NoneType' object has no attribute 'replace'

In [74]:
def optimal_params_parser(params_list):
    params = []
    values = []
    for param in params_list:
        print(param)
        _ = param.split(' ')
        params.append(_[0])
        print(_[-1].replace(',', ''))
        values.append(float(_[-1].replace(',', '')))
    return [params, values]

In [13]:
def extract_metadata(soup, experiment_id):
        
        
        experiment_exposure_route = soup.select('td.views-field-field-exposure-route')
        for route in experiment_exposure_route:
            exposure_route = route.get_text().strip()
        experiment_strain_container = soup.select('td.views-field-field-agent-strain')
        for strain in experiment_strain_container:
            experiment_strain = strain.get_text().strip()
        experiment_doseunits_container = soup.select('td.views-field-field-dose-units')
        for dose in experiment_doseunits_container:
            experiment_dose_units = dose.get_text().strip()
        experiment_host_container = soup.select('td.views-field-field-host-type')
        for host in experiment_host_container:
            experiment_host = host.get_text().strip()
        experiment_modeltype_container = soup.select('td.views-field-field-best-fit-model')
        for modeltype in experiment_modeltype_container:
            experiment_model = modeltype.get_text().strip()
        #TODO: Create this into a separate function
        experiment_params_container = soup.select('td.views-field-nothing')
        for param in experiment_params_container:
            experiment_optimal_params, experiment_optim_vals = optimal_params_parser(param.get_text().strip().split('\n'))
        ########
        experiment_response_cont = soup.select('td.views-field-field-response')
        for response in experiment_response_cont:
            experiment_response_type = response.get_text().strip()

        metadata_cols = ['Experiment Id', 'Exposure Route', 'Agent Strain', 'Dose Units', 'Host', 'Best Fit Model', f'Optimized Params {' '.join(experiment_optimal_params)}', 
                            'Response Type']
        metadata_data = [experiment_id, exposure_route, experiment_strain, experiment_dose_units, experiment_host,
                        experiment_model, experiment_optim_vals, experiment_response_type]
            
        metadata_df = pd.DataFrame([metadata_data], columns=metadata_cols)
        metadata_df.to_csv(f'{experiment_id}_metadata.csv')

In [23]:
#TODO: Have to adjust for pages that have the experiment data inside Experiment Dataset
def extract_from_dose_response_page(url, base_url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        #Getting experiment table data
        experimentId_container = soup.select('td.views-field-field-experiment-id-text')
        
        for container in experimentId_container:
            experiment_url = container.find('a').get('href')
            experiment_id = container.find('a').get_text()            
            extract_from_experiment_page(base_url + experiment_url, experiment_id)
            break
        
        #Getting experiment META data
        extract_metadata(soup, experiment_id)
        


In [68]:
def extract_datatable(data):
    
    soup = BeautifulSoup(data, 'html.parser')

    table = soup.find('table', {'border' : 1})
    data_arr = []
    for t in table:
       
       arr = t.get_text().strip().split('\n')
       
       
       if len(arr) > 1:
           arr = [a for a in arr if a != '']
           print(arr)
           k=0
           while k <= len(arr):
               data_arr.append(arr[k:k+4])
               k+=4
    data_arr = [item for item in data_arr if item !=[]]
    print(data_arr)
    

In [21]:
def extract_from_experiment_page(url, exp_id):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        experiment_dataset = soup.find_all(id='tablefield-0')

        for data in experiment_dataset:
            experiment_data_table = data.find_all('tr')
            if(len(experiment_data_table) > 0):
                print('Dose response has inline dataset')
                df = parse_meta_table(experiment_data_table, response.text)
                df.to_csv(f'{exp_id}_data.csv')
                #TODO: Add function here to extract the data if the dataset is embeded
            else:
                experiment_data_html = extract_datatable(response.text)


In [9]:
def parse_meta_table(data, page_txt):
    i = 0
    soup = BeautifulSoup(page_txt, 'html.parser')
    odds = soup.select('tr.odd')    
    evens = soup.select('tr.even')
    columns = soup.select('th.row_0')
    column_names = [column.get_text() for column in columns]
    rows = []
    even_counter = 2
    odd_counter = 1
    for even in evens:
        
        row = []
        for n in even.find_all(class_=f'row_{even_counter}'):
    
            row.append(n.get_text())
        even_counter += 2
        rows.append(row)
    for odd in odds:
        row = []
        
        for n in odd.find_all(class_=f'row_{odd_counter}'):
            
            row.append(n.get_text())
        rows.append(row)     
        odd_counter += 2
    df = pd.DataFrame(columns=column_names, data=rows) 
    return df.sort_values(['Dose (no. of organisms)'])
    
    
        

In [ ]:
#TODO: